# RAG 기본 구조 이해하기

## 1. 사전작업(Pre-processing) - 1~4 단계

![rag-1.png](./assets/rag-1.png)

![rag-1-graphic](./assets/rag-graphic-1.png)

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- 1단계 문서로드(Document Load): 문서 내용을 불러옵니다.
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

![rag-2.png](./assets/rag-2.png)

![](./assets/rag-graphic-2.png)

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. Dense: 유사도 기반 검색, Sparse: 키워드 기반 검색
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- 7단계 LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 환경설정


API KEY 를 설정합니다.


In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.


In [64]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


## RAG 기본 파이프라인(1~8단계)


In [90]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [105]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/어린왕자.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 43


In [106]:
print(docs[10].page_content)

7 장
다섯 째 날, 항상 양 덕분에, 어린 왕자의 비밀을 내가 하나 더 알게 되었지.
길게 생각해보더니 불쑥 약간 퉁명스런 말투로 서론 빼고 다짜고짜 묻더군.
"작은 나무를 먹는다면, 그럼 양들이 꽃들도 먹나요?" 
"그야 마주치면 먹겠지."
"심지어 꽃에 가시가 있어도요?" 
"그래, 꽃에 가시가 있어도 말이다." 
"그럼 가시는 뭐하려고 있는 건데요?"
난 알지 못했지. 
지금 엔진에 꼭 낀 볼트를 푸느라 여념이 없었거든. 
비행기 고장이 애초 생각보다 심각했던 지라 점점 무게감으로 다가왔고 무엇보다 목의 갈증은 더해
가는데 마실 물도 다 떨어져 가고 있었기 때문이야.
"그럼 가시는 뭐하려 있냐고요?"
어린 왕자는 내게 한 번 묻고 나면 포기하는 법이 없었어. 
그치만 난 지금 볼트를 푸느라 그런 게 귀에 와 닿지도 않았지.
"가시야 쓸모없지. 그건 꽃들이 짖궃어서니까!" 
"아!"
잠깐의 침묵 후 그 애가 일종의 적의까지 담아 내게 말하더구나.
"못 믿겠어요! 꽃들은 약해요. 순수하다고요. 그런 자신을 지킬 게, 그 애들이 생각하기에 무서운 
가시밖에 없는 거예요..."
난 대답하지 않았어. 
그냥 내 일에만 파묻혀 있고 싶었을 뿐이야. 
"이 볼트가 여간 단단하지 않은데. 아무래도 망치로 때려서라도 풀어야겠어." 
그때 어린 왕자가 또 나를 방해하더구나.
"아저씨도 꽃들이 그렇다고 생각하세요..."
"아니! 아니! 아니란다! 그냥 대답한 거야. 난 지금 바쁘잖니!" 
그 앤 어리벙벙한 표정을 지었어.
"바쁘다고요!"
손에 망치를 들고서 내 손가락에 묻어 반짝이는 시꺼먼 기름들을 보면서, 더구나 뭔가 엄청 못생긴 
물체에 몸을 숙이며 들여다보고 있는 나를 보면서 그 애가 말하더군. 
"어른들처럼 말하시는군요!"
이 부분에서 나도 살짝 놀랐단다. 
그치만 그 앤 무자비하게 이렇게까지 덧붙이더구나. 
"제 말을 혼동하고 계세요... 뒤죽박죽이라고요!"
그 앤 정말 화가 나 있었어. 
그 애의 머릿결이 바람에 나부끼는 게 보였을 정도니까.
"빨간 신사가 살던

`metadata` 를 확인합니다.

In [107]:
docs[10].__dict__

{'id': None,
 'metadata': {'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext',
  'creator': 'TextEdit',
  'creationdate': "D:20250715202247Z00'00'",
  'source': 'data/어린왕자.pdf',
  'file_path': 'data/어린왕자.pdf',
  'total_pages': 43,
  'format': 'PDF 1.3',
  'title': 'le petite prince',
  'author': '',
  'subject': '',
  'keywords': '',
  'moddate': "D:20250715202247Z00'00'",
  'trapped': '',
  'modDate': "D:20250715202247Z00'00'",
  'creationDate': "D:20250715202247Z00'00'",
  'page': 10},
 'page_content': '7 장\n다섯 째 날, 항상 양 덕분에, 어린 왕자의 비밀을 내가 하나 더 알게 되었지.\n길게 생각해보더니 불쑥 약간 퉁명스런 말투로 서론 빼고 다짜고짜 묻더군.\n"작은 나무를 먹는다면, 그럼 양들이 꽃들도 먹나요?" \n"그야 마주치면 먹겠지."\n"심지어 꽃에 가시가 있어도요?" \n"그래, 꽃에 가시가 있어도 말이다." \n"그럼 가시는 뭐하려고 있는 건데요?"\n난 알지 못했지. \n지금 엔진에 꼭 낀 볼트를 푸느라 여념이 없었거든. \n비행기 고장이 애초 생각보다 심각했던 지라 점점 무게감으로 다가왔고 무엇보다 목의 갈증은 더해\n가는데 마실 물도 다 떨어져 가고 있었기 때문이야.\n"그럼 가시는 뭐하려 있냐고요?"\n어린 왕자는 내게 한 번 묻고 나면 포기하는 법이 없었어. \n그치만 난 지금 볼트를 푸느라 그런 게 귀에 와 닿지도 않았지.\n"가시야 쓸모없지. 그건 꽃들이 짖궃어서니까!" \n"아!"\n잠깐의 침묵 후 그 

In [125]:

# 단계 2.1: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", ","],
    chunk_size=1000,
    chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)


In [126]:
# 단계 2.2: 문서 분할(Split Documents) using semanticchunking

text_splitter_sem = SemanticChunker(OpenAIEmbeddings())
split_documents_sem = text_splitter_sem.split_documents(docs) 


In [127]:
split_documents_sem[2].page_content

'보아 뱀\n여튼 어른들은 내게 충고하길 엉뚱한 보아 뱀이나 그리지 말고 지리, 역사, 샘(계산)이나 문법에 \n취미를 들어봐라. 그리하여 난 여섯 날 때 이미 화가의 꿈을 접어야 했지. 내 첫 그림과 두 번째 그림이 영 쓸모 없게 되자 낙담하고 말았거든. 나의 이런 일들에 대해 어른들은 물론 관심도 없었고 말이야. 설명을 해대는 아인 피곤하다는 투(말투)였었지. 그래서 다른 직업을 선택하게 된 거야. 그게 바로 하늘을 나는 비행사였지. 난 정말 전 세계를 날아다녔어. 그리 되니 지리학도 좀 도움이 되데. 난 중국이나 ‘애리조나’(미\n국의 주 이름)도 한눈에 첫 보면 알아보게 되었지. 물론 밤에 길을 잃을 때면 지리학에 대한 앎이 많은 도움이 되었고 말이야. 살아오며 다양한 사람들을 만났어, 대갠 심각한 어른들이였지.'

In [131]:
print (f"""
       500이상 문장 갯수: {
           len(
               [i for i in split_documents_sem if len(i.page_content) > 1000]
           )
       }""")


       500이상 문장 갯수: 2


In [132]:
# 단계 3: 임베딩(Embedding) 생성
import os  # For environment variables
import json  # For JSON operations
import chromadb  # For vector database operations
from chromadb.utils import embedding_functions  # For creating embedding functions

embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

# embeddings = embedding_functions.OpenAIEmbeddingFunction(
#         api_key=os.getenv("OPENAI_API_KEY"),
#         model_name="text-embedding-3-large"
#     )

In [133]:
embedded_query = embeddings.embed_query('What was the name mentioned in the coversation?')
len(embedded_query)

1536

In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=split_documents_sem,
    embedding=embeddings)

In [138]:
print(f"number of chunks: {len(split_documents_sem)}")
print(f"number saved in vecstoredb: {vectorstore.index.ntotal}")

number of chunks: 127
number saved in vecstoredb: 127


In [148]:
for doc in vectorstore.similarity_search("네번 째 별", k=1):
    print(doc.page_content, '\nend!!!!!!')

난 진지하다고. 세 번짼 언젠줄 아니... 바로 지금이야! 그러니까 내 말인즉슨 5
억 개란..."
"억이 뭔데요?"
장사꾼은 그제야 이 사태가 잠잠해질 기미가 없다는 걸 감 잡았지. "저 하늘에 때때로 반짝이는 게 보이지, 그게 억 개란다." 
"파리요?"
"아니, 반짝이는 작은 거 말이다."
"꿀벌요?"
"아니. 게으름뱅이도 웃게 만드는 저 작고 귀여운 황금빛깔 반짝이들 말이다. 난 지금 심각하다고! 장난칠 기분이 아니야."
"아! 별들요?"
"그래 별들 말이다."
"5억 개의 별들로 뭐하시게요?"
"정확하게 하자구나. 정확히 5억 162만 2,731개란다." 
"그러니까 이 많은 별들로 뭐하시는데요?"
"뭘 하냐고?"
"네."
"아무것도. 난 그저 저들을 소유하고 있을 뿐이다." 
"별들을 가지셨다고요?"
"그래."
"하지만 제가 만난 왕은..."
"왕들은 소유하진 않아. 그들은 통치할 뿐이지. 이것과 그것 별개의 문제라고."
"별들을 소유해서 어디다 써먹게요?"
"나를 부자로 만들어주지."
"어떻게 부자가 되신다는 건데요?"
"누가 나타나서 별들을 사줄 때마다 말이다."
"이 분도,"라며 어린 왕자가 속으로 말했어. "술꾼과 비슷하시군."
하지만 질문을 추가로 더 물었지. "별들을 어떻게 소유할 수 있는데요?"
"그럼 누구 건데? 대답해보렴, 심술쟁이야, 그러니 바로 나 장사꾼 거지."
"모르겠는데요. 누구건지는." 
end!!!!!!


In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장 (chromadb)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(
    documents=split_documents_sem,
    embedding=embeddings)

In [149]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.

In [150]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("어린왕자가 네번째 별에서 만난사람은 누구야?")

[Document(id='e6fa5678-9f1d-44c2-9fb5-d76e531ce4ce', metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'TextEdit', 'creationdate': "D:20250715202247Z00'00'", 'source': 'data/어린왕자.pdf', 'file_path': 'data/어린왕자.pdf', 'total_pages': 43, 'format': 'PDF 1.3', 'title': 'le petite prince', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20250715202247Z00'00'", 'trapped': '', 'modDate': "D:20250715202247Z00'00'", 'creationDate': "D:20250715202247Z00'00'", 'page': 15}, page_content='그렇지 \n않음 누가 절 찾아오겠어요? 당신은 떠나실 거죠. 야수들에 대해선 걱정 붙들어 매세요. 제겐 가시\n가 있잖아요."\n그렇게 그녀는 순진한 표정으로 자신의 네(4개) 가시들을 보여주더래. 그런 다음 그녀가 덧붙였지. "오래 있지 마세요, 헷갈린다고요. 가시려면 어서 가버리세요."\n왜냐하면 그녀는 자신이 우는 모습을 어린 왕자에게만은 보여주고 싶지 않았던 게야. 그토록 자존심이 강한 꽃이었던 것이지... 10 장\n어린 왕자의 별 가까이로 소행성 325호, 326호, 327호, 328호, 329호와 330호가 있었데. 그래서 이 기회에 그 별들을 방문해보기로 마음 먹고 떠난 거였지. 어린 왕자가 간 첫 번째 별엔 왕이 살고 있었데. 왕은 ‘자주빛’(옛날부터 황제를 상징하는 색이 자주색임. 자주색 옷은 아무나 입을 수 있는 옷의 \n색깔이 아님) 옷과 (왕이 입는) 가운을 입고서 홀로 그렇게 홀로 그러면서도 근엄한 왕좌(왕의 

In [173]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
우선적으로 검색된 다음 문맥(context) 만을 사용하여 질문(question) 에 답하세요. 문서에 직접적인 설명이없더라도, 문맥상으로 유추를 해보고 생각을 곁들여도 괜찮아. 그리고도 답을 모른다면 '잘 모르겠습니다' 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.




#Context: 
{context}

#Question:
{question}

#Answer:"""
)



In [159]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0)

In [160]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

In [168]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 각각의 별에서 만난사람들을 나열해줘"
response = chain.invoke(question)
print(response)

주인공인 어린 왕자가 각각의 별에서 만난 사람들은 다음과 같습니다.

1. 첫 번째 별: 왕
2. 두 번째 별: 허영쟁이
3. 세 번째 별: 술꾼
4. 네 번째 별: 사업가(장사꾼)
5. 다섯 번째 별: 가로등지기
6. 여섯 번째 별: 지리학자(엄청 큰 책을 쓰고 있는 노신사)

(문맥에 등장하는 인물들을 기준으로 나열하였습니다.)


In [187]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 도착한 두 번째 별에선 어떤 감정이었어?"
response = chain.invoke(question)
print(response)

주인공인 어린 왕자는 두 번째 별(자신의 별)이 아주 작다는 사실을 알게 되었을 때 살짝 구슬퍼지는 감정을 느꼈던 것 같습니다. "곧장 가도 얼마 못 간다고요..."라고 말하며 자신의 집이 아주 작다는 점에 대해 약간 슬퍼하는 듯한 모습을 보였습니다.


In [175]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response = chain.invoke(question)
print(response)

주어진 문맥에서는 주인공이 별들에서 만난 사람들 중 누구를 가장 싫어하는지에 대해 직접적으로 언급하고 있지 않습니다. 하지만 21페이지의 내용을 보면, 어린 왕자가 별들을 소유하려는 장사꾼(사업가)과의 대화에서 여간 만족스럽지 못했다고 표현되어 있습니다. 이로 미루어 볼 때, 어린 왕자는 별들을 소유하고 장사하려는 장사꾼(사업가)을 그다지 좋아하지 않는 것으로 보입니다. 따라서 가장 싫어하는 인물은 별을 소유하려는 장사꾼(사업가)일 가능성이 높다고 생각합니다.


In [176]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 두번째 별을 떠난 이유가 뭐라고 생각해?"
response = chain.invoke(question)
print(response)

주인공인 어린 왕자가 두 번째 별을 떠난 이유는, 그 별에 사는 허영심쟁이와 친구가 되고 싶지 않았기 때문이라고 볼 수 있습니다. 문맥에 따르면 어린 왕자는 각 별에서 만난 인물들과의 관계나 상황에 만족하지 못해 계속해서 다음 별로 이동합니다. 두 번째 별의 허영심쟁이는 자신만을 칭찬해주길 바라는 인물이었기 때문에, 어린 왕자는 그와 진정한 친구가 될 수 없다고 느껴 별을 떠난 것으로 보입니다.


In [213]:
# 단계 5: 검색기(Retriever) 생성. 고급

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-2025-04-14", temperature=0)


retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)


INFO:langchain.retrievers.multi_query:Generated queries: ['주인공이 별들에서 만난 사람들 중에서 가장 부정적으로 평가한 인물은 누구인가요?  ', '어린 왕자가 여러 별을 여행하며 만난 인물들 중에서 주인공이 가장 싫어한 사람은 누구일까요?  ', '주인공이 별을 여행하면서 만난 인물들 중에서 가장 반감을 느낀 대상은 누구라고 할 수 있나요?']


In [214]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [215]:

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "주인공이 별들에서 만난사람중 가장 싫어하는 인물은 누구라고 생각해?"
response = chain.invoke(question)


INFO:langchain.retrievers.multi_query:Generated queries: ['주인공이 별들에서 만난 인물들 중에서 가장 부정적으로 평가한 사람은 누구인가요?  ', '주인공이 여러 별을 여행하며 만난 사람들 중에서 가장 싫어했던 인물은 누구라고 볼 수 있을까요?  ', '주인공이 별들에서 만난 인물들 중에서 가장 반감을 느낀 대상은 누구인지 알려주세요.']


In [218]:

response

'주어진 문맥을 보면, 어린 왕자가 별들에서 만난 여러 인물 중에서 "장사꾼"에 대해 특히 비판적이고 만족스럽지 못한 대화를 나누는 장면이 나옵니다. 장사꾼은 별들을 소유하려 하고, 그것을 부의 상징으로 여기며, 별들을 세는 일에만 집착합니다. 어린 왕자는 이런 태도에 대해 "술꾼과 비슷하시군"이라고 속으로 생각하며, 별들을 소유한다는 개념 자체를 이해하지 못하고, 대화에 만족하지 못합니다.\n\n이러한 맥락에서 볼 때, 어린 왕자가 별들에서 만난 사람들 중 가장 싫어하는 인물은 "장사꾼"일 가능성이 높다고 생각합니다.'

## 전체 코드

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/le_Petit_Prince.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

ValueError: File path data/SPRI_AI_Brief_2023년12월호_F.pdf is not a valid file or url

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)